In [8]:
import pandas as pd
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Load the dataset
data_path = '/workspaces/Final-Year-Project/Cleaned Data/TrainTestData.csv'
df = pd.read_csv(data_path)

print(f"🔍 AB - Original dataset shape: {df.shape}")

# Drop rows with NaN in any feature column
df = df.dropna()

print(f"🧹 AB - After dropping NaNs: {df.shape}")

# Define features and target
X = df.drop('Diabetes Status', axis=1)
y = df['Diabetes Status']

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"📊 AB - Final dataset shape after scaling: {X_scaled.shape}")

# Initialize the AdaBoost model
ab = AdaBoostClassifier(n_estimators=100, random_state=42)

# Set up K-Fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation using accuracy as the metric
ab_kfold_scores = cross_val_score(ab, X_scaled, y, cv=kf, scoring='accuracy')

# Fit the model on the full dataset for classification report & confusion matrix
ab.fit(X_scaled, y)
y_pred = ab.predict(X_scaled)

# Print results in the same format as previous models
print(f"✅ AB - K-Fold Mean Accuracy: {ab_kfold_scores.mean():.4f}")
print("📌 AB Classification Report:")
print(classification_report(y, y_pred))
print("🟦 AB Confusion Matrix:")
print(confusion_matrix(y, y_pred))


🔍 AB - Original dataset shape: (4435, 10)
🧹 AB - After dropping NaNs: (3003, 10)
📊 AB - Final dataset shape after scaling: (3003, 9)
✅ AB - K-Fold Mean Accuracy: 0.6886
📌 AB Classification Report:
              precision    recall  f1-score   support

           0       0.70      0.55      0.62      1339
           1       0.69      0.81      0.75      1664

    accuracy                           0.70      3003
   macro avg       0.70      0.68      0.68      3003
weighted avg       0.70      0.70      0.69      3003

🟦 AB Confusion Matrix:
[[ 740  599]
 [ 316 1348]]


In [9]:
from sklearn.metrics import classification_report, confusion_matrix

# File path for validation data
valid_path = "/workspaces/Final-Year-Project/Cleaned Data/ValidationData.csv"

### 🎯 VALIDATION PHASE ###
# Load validation data
df_valid = pd.read_csv(valid_path)
df_valid = df_valid.dropna()

# Define features and target
X_valid = df_valid.drop('Diabetes Status', axis=1)
y_valid = df_valid['Diabetes Status']

# Standardize validation data (using the same scaler from training)
X_valid_scaled = scaler.transform(X_valid)

print(f"🧪 AB - Validation dataset shape: {X_valid.shape}")

# Evaluate on validation set
y_pred_valid = ab.predict(X_valid_scaled)

# Classification Report
print(f"📌 AB - Validation Classification Report:")
print(classification_report(y_valid, y_pred_valid))

# Confusion Matrix
print("🟦 AB - Validation Confusion Matrix:")
print(confusion_matrix(y_valid, y_pred_valid))


🧪 AB - Validation dataset shape: (335, 9)
📌 AB - Validation Classification Report:
              precision    recall  f1-score   support

           0       0.66      0.62      0.64       145
           1       0.72      0.76      0.74       190

    accuracy                           0.70       335
   macro avg       0.69      0.69      0.69       335
weighted avg       0.70      0.70      0.70       335

🟦 AB - Validation Confusion Matrix:
[[ 90  55]
 [ 46 144]]
